# Coding Exercise

You must correctly implement the function described in the prompt below.

Feel free to test out pieces of code to help you write the solution.

Please thoroughly test that the final code implements the function correctly.

## Prompt

**Function signature:** `getKth(n: int, k: int) -> List[int]`

    There is a square on a plane with (0,0) as its lower left and (2^N+2, 2^N+2) as its upper right corner. It is known that K distinct points have to be painted. Each of them should be strictly inside the square and should have integral coordinates. The first point to be painted is always (1,1). For each of the next points, the one from which the Euclidean distance to the closest already painted point is maximum should be chosen. In case of ties, the leftmost point should be painted. If there are still several possibilities, the lowermost point is chosen. Return List[int] with exactly two elements, the X and Y coordinates of the K-th point to be painted.
    Notes
    -The Euclidean distance between two points (x1,y1) and (x2,y2) is equal to the square root of (x1-x2)^2+(y1-y2)^2.
 
    Constraints
    -N will be between 2 and 10, inclusive.
    -K will be between 1 and the amount of points inside the square with side length 2^N+2, inclusive.
 
    Examples
    0)
        4
    2

    Returns: {17, 17 }
    The square stretches from (0,0) to (18,18). After you paint (1,1), the farthest point within the square is (17,17).

    1)
        4
    3

    Returns: {1, 17 }
    Now there are two candidates. Both upper-left and bottom-right points within the square are equally distant from the already painted two, so we choose the leftmost one.

    2)
        4
    5

    Returns: {9, 9 }
    After you paint all the corners, the best choice is the center of the square.

    3)
        3
    14

    Returns: {1, 3 }

    4)
        5
    1089

    Returns: {33, 32 }

    

Can we solve this by brute force?
The maximum square size is $ \text{size} = 2^{10}+2 \approx 10^3$,
therefore the number of points in the square will be of order $10^6$.
If we need to calculate all the distances for each iteration, we will need to do at least $\text{size}^3 \approx 10^9$ operations, hence brute force is not viable.

Given that the inside of the square has odd width/height, the problem seems to have symmetry. Can we use it to find a better algorithm?

Let us draw a few patterns

For N = 2, the first few painted squares are:

| | | | | |
|-|-|-|-|-|
|3|.|8|.|2|
|.|.|.|.|.|
|6|.|5|.|9|
|.|.|.|.|.|
|1|.|7|.|4|

For N = 3, the first few painted squares are:

| | | | | | | | | |
|-|-|-|-|-|-|-|-|-|
|3|.|.|.|.|.|.|.|2|
|.|.|.|.|.|.|.|.|.|
|.|.|.|.|.|.|.|.|.|
|.|.|.|.|.|.|.|.|.|
|.|.|.|.|.|.|.|.|.|
|.|.|.|.|.|.|.|.|.|
|.|.|.|.|.|.|.|.|.|
|.|.|.|.|.|.|.|.|.|
|1|.|.|.|.|.|.|.|4|


next is the middle

| | | | | | | | | |
|-|-|-|-|-|-|-|-|-|
|3|.|.|.|.|.|.|.|2|
|.|.|.|.|.|.|.|.|.|
|.|.|.|.|.|.|.|.|.|
|.|.|.|.|.|.|.|.|.|
|.|.|.|.|5|.|.|.|.|
|.|.|.|.|.|.|.|.|.|
|.|.|.|.|.|.|.|.|.|
|.|.|.|.|.|.|.|.|.|
|1|.|.|.|.|.|.|.|4|

after this we will paint

| | | | | | | | | |
|-|-|-|-|-|-|-|-|-|
|3|.|.|.|8|.|.|.|2|
|.|.|.|.|.|.|.|.|.|
|.|.|.|.|.|.|.|.|.|
|.|.|.|.|.|.|.|.|.|
|6|.|.|.|5|.|.|.|9|
|.|.|.|.|.|.|.|.|.|
|.|.|.|.|.|.|.|.|.|
|.|.|.|.|.|.|.|.|.|
|1|.|.|.|7|.|.|.|4|

Next, it is the middle if each subsquare, for example

| | | | | |
|-|-|-|-|-|
|6|.|. |.|5|
|.|.|. |.|.|
|.|.|10|.|.|
|.|.|. |.|.|
|1|.|. |.|7|

And the pattern repeats itself once again!

The ideia is to generate this fractal pattern to order all the points.
The points will be orderded by a layer number (points at the same distance), and them by smallest x and y.

In [5]:
from pandas import DataFrame

def mainsquare(N, k):
    #generates the first 4 steps
    
    size = 2**N+2
    
    if k == 1:
        return 1, 1
    elif k == 2:
        return size-1, size-1
    elif k == 3:
        return 1, size-1
    elif k == 4:
        return size-1, 1
    
    #call subsquare to generate the pattern 
    k -= 4
    df = DataFrame(subsquare(N, k, 0, 0), columns=['layer', 'x', 'y'])
    df = df.sort_values(by=['layer', 'x', 'y']) #sort by layer, x, and y
    return df
    
def subsquare(N, k, x, y, layer=1):
    #generates squares pattern
    
    #if N == 0 all points were selected
    if N == 0:
        return [[]]
    
    #results
    res = []
    
    #square size
    size = 2**N+2
    
    #the points in a given iteration
    res.append([layer, x+size//2, y+size//2]) #middle 
    res.append([layer, x+1      , y+size//2]) #left
    res.append([layer, x+size//2, y+1])       #bottom middle
    res.append([layer, x+size//2, y+size-1])  #top middle
    res.append([layer, x+size-1 , y+size//2]) #right
    
    layer += 1 #advances 1 layer
    size //= 2 #subsquare size
    res += subsquare(N-1, k, x       , y       , layer) #call with N-1
    res += subsquare(N-1, k, x+size-1, y       , layer) #and give a offset x,y to calculate the coordinates
    res += subsquare(N-1, k, x       , y+size-1, layer) #of the subsquare
    res += subsquare(N-1, k, x+size-1, y+size-1, layer)
    
    return res

mainsquare(2, 10)

,layer,x,y
1,1.0,1.0,3.0
2,1.0,3.0,1.0
0,1.0,3.0,3.0
3,1.0,3.0,5.0
4,1.0,5.0,3.0
6,2.0,1.0,2.0
24,2.0,1.0,4.0
7,2.0,2.0,1.0
5,2.0,2.0,2.0
8,2.0,2.0,3.0


There are a lot of NaNs that appear when we return [[]]. This will be solved by adding a dropna to the dataframe.
Theare also duplicates that need to be dropped (duplicates appear at the intersection of the subsquares).

In [6]:
def mainsquare(N, k):
    #generates the first 4 steps
    
    size = 2**N+2
    
    if k == 1:
        return 1, 1
    elif k == 2:
        return size-1, size-1
    elif k == 3:
        return 1, size-1
    elif k == 4:
        return size-1, 1
    
    #call subsquare to generate the pattern 
    k -= 4
    df = DataFrame(subsquare(N, k, 0, 0), columns=['layer', 'x', 'y'])
    df = df.drop_duplicates()
    df = df.dropna()
    df = df.sort_values(by=['layer', 'x', 'y'])
    return df

mainsquare(2, 10)

,layer,x,y
1,1.0,1.0,3.0
2,1.0,3.0,1.0
0,1.0,3.0,3.0
3,1.0,3.0,5.0
4,1.0,5.0,3.0
6,2.0,1.0,2.0
24,2.0,1.0,4.0
7,2.0,2.0,1.0
5,2.0,2.0,2.0
8,2.0,2.0,3.0


The order of the first 4 points is incorrect.
I will solve this by adding them in individual, sequential layers.

In [9]:
def mainsquare(N, k):
    #generates the first 4 steps
    
    res = []
    
    size = 2**N+2
    
    res.append([0, 1, 1])
    res.append([1, size-1, size-1])
    res.append([2, 1, size-1])
    res.append([3, size-1, 1])
    
    res += subsquare(N, k, 0, 0, layer=4)
    
    #call subsquare to generate the pattern 
    df = DataFrame(res, columns=['layer', 'x', 'y'])
    df = df.drop_duplicates()
    df = df.dropna()
    df = df.sort_values(by=['layer', 'x', 'y'])
    return df

mainsquare(2, 10)

,layer,x,y
0,0.0,1.0,1.0
1,1.0,5.0,5.0
2,2.0,1.0,5.0
3,3.0,5.0,1.0
5,4.0,1.0,3.0
6,4.0,3.0,1.0
4,4.0,3.0,3.0
7,4.0,3.0,5.0
8,4.0,5.0,3.0
10,5.0,1.0,2.0


The middle of the square should come next. I will add the middle in a separate layer.

In [10]:
def subsquare(N, k, x, y, layer=1):
    #generates squares pattern
    
    #if N == 0 all points were selected
    if N == 0:
        return [[]]
    
    #results
    res = []
    
    size = 2**N+2
    
    #middle of the subsquare
    res.append([layer, x+size//2, y+size//2])
    
    #points between the middle and the edges
    layer += 1
    res.append([layer, x+1      , y+size//2])
    res.append([layer, x+size//2, y+1])
    res.append([layer, x+size//2, y+size-1])
    res.append([layer, x+size-1 , y+size//2])
    
    #call the next layer
    layer += 1
    size //= 2
    res += subsquare(N-1, k, x       , y       , layer)
    res += subsquare(N-1, k, x+size-1, y       , layer)
    res += subsquare(N-1, k, x       , y+size-1, layer)
    res += subsquare(N-1, k, x+size-1, y+size-1, layer)
    
    return res

mainsquare(2, 10)

,layer,x,y
0,0.0,1.0,1.0
1,1.0,5.0,5.0
2,2.0,1.0,5.0
3,3.0,5.0,1.0
4,4.0,3.0,3.0
5,5.0,1.0,3.0
6,5.0,3.0,1.0
7,5.0,3.0,5.0
8,5.0,5.0,3.0
9,6.0,2.0,2.0


We do not need K in mainsquare and subsquare. Let me remove them, and implement the final solution.

In [15]:
from typing import List

def mainsquare(N):
    #generates the first 4 steps
    
    res = []
    
    size = 2**N+2
    
    res.append([0, 1, 1])
    res.append([1, size-1, size-1])
    res.append([2, 1, size-1])
    res.append([3, size-1, 1])
    
    res += subsquare(N, 0, 0, layer=4)
    
    #call subsquare to generate the pattern 
    df = DataFrame(res, columns=['layer', 'x', 'y'])
    df = df.drop_duplicates()
    df = df.dropna()
    df = df.sort_values(by=['layer', 'x', 'y'])
    return df

def subsquare(N, x, y, layer=1):
    #generates squares pattern
    
    #if N == 0 all points were selected
    if N == 0:
        return [[]]
    
    res = []
    
    size = 2**N+2
    
    #middle of the subsquare
    res.append([layer, x+size//2, y+size//2])
    
    #points between the middle and the edges
    layer += 1
    res.append([layer, x+1      , y+size//2])
    res.append([layer, x+size//2, y+1])
    res.append([layer, x+size//2, y+size-1])
    res.append([layer, x+size-1 , y+size//2])
    
    #call the next layer
    layer += 1
    size //= 2
    res += subsquare(N-1, x       , y       , layer)
    res += subsquare(N-1, x+size-1, y       , layer)
    res += subsquare(N-1, x       , y+size-1, layer)
    res += subsquare(N-1, x+size-1, y+size-1, layer)
    
    return res

def getKth(n: int, k: int) -> List[int]:
    
    #generate points in sequence
    df = mainsquare(n).reset_index(drop=True)
    
    #get k-th
    x, y = df.loc[k-1, 'x'], df.loc[k-1, 'y']
    
    return [x, y]

getKth(4, 2)

[17.0, 17.0]

The results above should be [17, 17] 

In [16]:
getKth(4, 3)

[1.0, 17.0]

The results above should be [1, 17] 

In [17]:
getKth(4, 5)

[9.0, 9.0]

The results above should be [9, 9] 

In [18]:
getKth(3, 14)

[1.0, 3.0]

The result above should be [1, 3] 

In [19]:
getKth(5, 1089)

[33.0, 32.0]

The result above should be [33, 32].

Let me test the worst case scenario.

In [21]:
from datetime import datetime

t1 = datetime.now()

size = 2**10+2

res = getKth(10, (size-1)**2)
print(res)
print(f'should be {size-1}, {size-2}')

t2 = datetime.now()
print(t2 - t1)

[1025.0, 1024.0]
should be 1025, 1024
0:00:04.503067


The execution time is fine and the result is correct. 